In [1]:
#importation du fichier csv
import pandas as pd
import random
df=pd.read_csv("opinions.csv")

In [2]:
#supprimer la colonne review:
df = df.drop('review', axis=1)


In [3]:
#affichage de nombre de valeurs vides par colonne:
missing_values = df.isnull().sum()
missing_df = pd.DataFrame(missing_values, columns=['Nombre de valeurs vides'])
print(missing_df)


         Nombre de valeurs vides
ID                             0
score                          3
opinion                        0


In [4]:
#matrice des valeurs vides:
import numpy as np
missing_matrix = df.isnull().to_numpy()
print(missing_matrix)
np.savetxt('missing_matrix.txt', missing_matrix, fmt='%d')

[[False False False]
 [False False False]
 [False False False]
 ...
 [False False False]
 [False False False]
 [False False False]]


In [5]:
#création de la colonne ville:
villes = [random.choice(['1', '2', '3', '4', '5']) for _ in range(len(df))]
df['ville']=villes
df

,ID,score,opinion,ville
0,3870,0.5000,positive,5
1,62,0.2794,positive,2
2,487,0.1827,positive,2
3,3204,0.3682,positive,2
4,1265,0.2333,positive,2
...,...,...,...,...
4079,852,0.4571,positive,4
4080,2163,0.0000,neutral,1
4081,2488,0.0000,neutral,3
4082,651,0.0000,neutral,5


In [6]:
# Calculer la moyenne de la colonne
moyenne = df["score"].mean()

# Remplacer les valeurs manquantes de la colonne par la moyenne
df["score"].fillna(moyenne, inplace=True)

In [7]:
missing_values = df.isnull().sum()
print(missing_values)

ID         0
score      0
opinion    0
ville      0
dtype: int64


In [8]:
import pandas as pd
from sklearn.impute import KNNImputer


imputer = KNNImputer(n_neighbors=5)

# Sélectionner uniquement la colonne "score" pour l'imputation
score_column = df['score'].values.reshape(-1, 1)

# Appliquer l'imputation KNN aux valeurs manquantes de la colonne "score"
score_imputed = imputer.fit_transform(score_column)

# Remplacer les valeurs manquantes dans la colonne "score" par les valeurs imputées
df['score'] = score_imputed

print(df)

        ID   score   opinion ville
0     3870  0.5000  positive     4
1       62  0.2794  positive     5
2      487  0.1827  positive     1
3     3204  0.3682  positive     1
4     1265  0.2333  positive     3
...    ...     ...       ...   ...
4079   852  0.4571  positive     3
4080  2163  0.0000   neutral     4
4081  2488  0.0000   neutral     3
4082   651  0.0000   neutral     5
4083   868 -0.3125  negative     5

[4084 rows x 4 columns]


In [12]:
def similarity(row1, row2):
    # Calculer la similarité selon le critère du score
    score_similarity = np.abs(row1['score'] - row2['score']) *2.5

    # Calculer la similarité selon le critère de l'opinion
    opinion_similarity = 0
    if row1['opinion'] == row2['opinion']:
        opinion_similarity = 2
    elif (row1['opinion'] == 'neutral' and row2['opinion'] == 'postive') or (row1['opinion'] == 'neutral' and row2['opinion'] == 'negative'):
        opinion_similarity = 1
    
        

    # Calculer la similarité selon le critère de la ville
    ville_similarity = 0
    if row1['ville'] == row2['ville']:
        ville_similarity = 0.5

    # Calculer la similarité totale
    total_similarity = score_similarity + opinion_similarity + ville_similarity

    return total_similarity

# Fonction pour calculer la matrice de similarité entre les clients
def similarity_matrix(data):
    # Initialiser la matrice de similarité
    n = data.shape[0]
    similarity_matrix = np.zeros((n, n))

    # Calculer la similarité entre chaque paire de clients
    for i in range(n):
        for j in range(i, n):
            if i == j:
                similarity_matrix[i, j] = 0
            else:
                similarity_matrix[i, j] = similarity(data.iloc[i], data.iloc[j])
                similarity_matrix[j, i] = similarity_matrix[i, j]

    return similarity_matrix



print(df)

# Calculer la matrice de similarité et copie dans un fichier CSV
df_100 = df.head(200)
# df_100=df
print(type(df_100))

similarity_matrix = similarity_matrix(df_100)
print(similarity_matrix)
np.savetxt('premierematrice_200.csv', similarity_matrix, delimiter='\t')
print(similarity_matrix.shape)

        ID   score   opinion ville
0     3870  0.5000  positive     5
1       62  0.2794  positive     2
2      487  0.1827  positive     2
3     3204  0.3682  positive     2
4     1265  0.2333  positive     2
...    ...     ...       ...   ...
4079   852  0.4571  positive     4
4080  2163  0.0000   neutral     1
4081  2488  0.0000   neutral     3
4082   651  0.0000   neutral     5
4083   868 -0.3125  negative     2

[4084 rows x 4 columns]
<class 'pandas.core.frame.DataFrame'>
[[0.      2.5515  2.79325 ... 2.125   1.25    2.25   ]
 [2.5515  0.      2.74175 ... 2.0735  0.6985  2.8015 ]
 [2.79325 2.74175 0.      ... 1.83175 0.45675 3.04325]
 ...
 [2.125   2.0735  1.83175 ... 0.      0.875   2.375  ]
 [1.25    0.6985  0.45675 ... 0.875   0.      1.5    ]
 [2.25    2.8015  3.04325 ... 2.375   1.5     0.     ]]
(200, 200)


# Partie2:



In [14]:
#importation du fichier csv et suppression des colonnes score et opinions:
import pandas as pd
import random
col=pd.read_csv("opinions.csv")
col = col.drop('score', axis=1)
col = col.drop('opinion', axis=1)
col.head()

,ID,review
0,3870,able play youtube alexa
1,62,+A35able recognize indian accent really well d...
2,487,absolute smart device amazon connect external ...
3,3204,absolutely amaze new member family control hom...
4,1265,absolutely amaze previously sceptical invest m...


In [15]:
import re

def nettoyer_texte(texte):
    texte = re.sub(r'[^\w\s]', '', texte)  # Supprimer les caractères spéciaux
    texte = re.sub(r'[\d+]', '', texte)  # Supprimer les numéros
    texte = texte.lower()  # Mettre tout en minuscules
    return texte


# Application de la fonction de nettoyage à la colonne "review"
col['review'] = col['review'].apply(nettoyer_texte)

# Afficher le DataFrame mis à jour
print(col)
col.to_csv('data_version2.csv', index=False)

        ID                                             review
0     3870                            able play youtube alexa
1       62  aable recognize indian accent really well drop...
2      487  absolute smart device amazon connect external ...
3     3204  absolutely amaze new member family control hom...
4     1265  absolutely amaze previously sceptical invest m...
...    ...                                                ...
4079   852  yo yo yo love go if want one smart speaker val...
4080  2163                                      youtube music
4081  2488  youtube support nahi kartasong recognise achha...
4082   651  yup proscontrols wipro light amazinglysony bra...
4083   868  zero integration capabilities fire tv devices ...

[4084 rows x 2 columns]


In [16]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

nltk.download('vader_lexicon')  # Téléchargement du lexique de sentiment
nltk.download('wordnet')  # Téléchargement du corpus WordNet pour la lemmatisation
nltk.download('averaged_perceptron_tagger')

# Initialisation de l'analyseur de sentiment et du lemmatiseur
sia = SentimentIntensityAnalyzer()
lemmatizer = WordNetLemmatizer()

# Fonction pour déterminer l'opinion en fonction du score
def get_opinion(score):
    if score > 0.1:
        return 'positive'
    elif score < -0.1:
        return 'negative'
    else:
        return 'neutral'

# Fonction pour appliquer la lemmatisation à un texte donné
def lemmatize_text(text):
    lemmatized_text = ' '.join([lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in text.split()])
    return lemmatized_text

# Fonction pour obtenir la partie du discours WordNet correspondante
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN)

# Chargement du DataFrame
df = pd.read_csv('data_version2.csv')  # Remplacez 'your_dataframe.csv' par le chemin vers votre DataFrame

# Application de la lemmatisation à la colonne 'review'
df['lemmatized_review'] = df['review'].apply(lemmatize_text)

# Application de l'analyse de sentiment à la colonne 'lemmatized_review' pour obtenir le score et l'opinion
df['score'] = df['lemmatized_review'].apply(lambda x: sia.polarity_scores(x)['compound'])
df['opinion'] = df['score'].apply(get_opinion)

# Enregistrement du DataFrame mis à jour avec le score de sentiment et l'opinion
df.to_csv('updated_dataframe.csv', index=False)  # Remplacez 'updated_dataframe.csv' par le chemin et le nom de fichier souhaités


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [17]:
#calcule de la matrice de similarité globale du la data_version2:
import pandas as pd
from itertools import product
df1 = pd.read_csv('updated_dataframe.csv')
# Sélectionner un petit échantillon de votre dataframe
sample_size = 100  # Spécifiez la taille de l'échantillon souhaitée
df_sample = df1.sample(n=sample_size)

# Calculer la similarité basée sur les colonnes "score" et "opinion"
def calculate_similarity(row1, row2):
    score_sim = 2.5 if row1['score'] == row2['score'] else 0
    opinion_sim = 2 if row1['opinion'] == row2['opinion'] else 0
    if row1['opinion'] == 'Neutral' or row2['opinion'] == 'Neutral':
        opinion_sim = 1
    
    total_sim = score_sim + opinion_sim
    return total_sim

# Créer une matrice de similarité
def create_similarity_matrix(df):
    index_combinations = list(product(df.index, repeat=2))
    similarity_scores = [calculate_similarity(df.loc[i], df.loc[j]) for i, j in index_combinations]
    
    similarity_matrix = pd.DataFrame(index=df.index, columns=df.index, 
                                     data=np.array(similarity_scores).reshape(len(df), len(df)))
    return similarity_matrix

# Obtenir la matrice de similarité pour l'échantillon sélectionné
similarity_matrix = create_similarity_matrix(df_sample)

# Afficher la matrice de similarité
print(similarity_matrix)
np.savetxt('deuxièmematrice_100.csv', similarity_matrix, delimiter='\t')


      1733  58    3400  2855  75    517   713   2536  3194  3670  ...  1387  \
1733   4.5   2.0   0.0   2.0   2.0   2.0   0.0   2.0   2.0   2.0  ...   2.0   
58     2.0   4.5   0.0   2.0   2.0   2.0   0.0   2.0   2.0   2.0  ...   2.0   
3400   0.0   0.0   4.5   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
2855   2.0   2.0   0.0   4.5   2.0   2.0   0.0   2.0   2.0   2.0  ...   2.0   
75     2.0   2.0   0.0   2.0   4.5   2.0   0.0   2.0   2.0   2.0  ...   2.0   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
4008   0.0   0.0   0.0   0.0   0.0   0.0   2.0   0.0   0.0   0.0  ...   0.0   
790    0.0   0.0   0.0   0.0   0.0   0.0   2.0   0.0   0.0   0.0  ...   0.0   
1752   0.0   0.0   0.0   0.0   0.0   0.0   2.0   0.0   0.0   0.0  ...   0.0   
2664   0.0   0.0   4.5   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
1339   2.0   2.0   0.0   2.0   2.0   2.0   0.0   2.0   2.0   2.0  ...   4.5   

      1638  1225  3768  481   4008  790   1752  266

# pour aller plus loin dans la partie 2:


In [ ]:
import pandas as pd
from transformers import BertTokenizerFast, BertForSequenceClassification, TrainingArguments, Trainer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

# Définition de la classe ReviewDataset qui hérite de la classe Dataset
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Chargement des données
df_with_labels = pd.read_csv('opinions.csv').sample(frac=0.1)  # utiliser une fraction des données
df_without_labels = pd.read_csv('clean.csv').sample(frac=0.1)  # utiliser une fraction des données

# Initialisation du tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

# Tokenization du texte
encodings = tokenizer(df_with_labels['review'].tolist(), truncation=True, padding=True)

# Encodage des labels avec LabelEncoder
label_encoder_opinion = LabelEncoder()
df_with_labels['opinion'] = label_encoder_opinion.fit_transform(df_with_labels['opinion'])

label_encoder_score = LabelEncoder()
df_with_labels['score'] = label_encoder_score.fit_transform(df_with_labels['score'])

# Encodage one-hot des labels
one_hot_encoder = OneHotEncoder(sparse=False)
labels_opinion_onehot = one_hot_encoder.fit_transform(df_with_labels['opinion'].values.reshape(-1, 1))
labels_score_onehot = one_hot_encoder.transform(df_with_labels['score'].values.reshape(-1, 1))

# Création des jeux de données
dataset_opinion = ReviewDataset(encodings, labels_opinion_onehot)
dataset_score = ReviewDataset(encodings, labels_score_onehot)

# Initialisation des modèles
model_opinion = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=3)
model_score = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=3)

# Initialisation des arguments d'entraînement
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Initialisation des entraîneurs
trainer_opinion = Trainer(
    model=model_opinion,
    args=training_args,
    train_dataset=dataset_opinion,
)

trainer_score = Trainer(
    model=model_score,
    args=training_args,
    train_dataset=dataset_score,
)

# Entraînement des modèles
trainer_opinion.train()
trainer_score.train()

# Tokenization et prédiction
new_reviews = df_without_labels['review'].tolist()
new_encodings = tokenizer(new_reviews, truncation=True, padding=True)

# Prédiction des opinions
new_dataset_opinion = ReviewDataset(new_encodings, labels=[0]*len(new_reviews))
preds_opinion = trainer_opinion.predict(new_dataset_opinion)
df_without_labels['predicted_opinion'] = label_encoder_opinion.inverse_transform(preds_opinion.predictions.argmax(axis=1))

# Prédiction des scores
new_dataset_score = ReviewDataset(new_encodings, labels=[0]*len(new_reviews))
preds_score = trainer_score.predict(new_dataset_score)
df_without_labels['predicted_score'] = label_encoder_score.inverse_transform(preds_score.predictions.argmax(axis=1))

# Sauvegarde des prédictions
df_without_labels.to_csv('predictions.csv', index=False)
